In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
# forex_data = pd.read_csv('/content/drive/MyDrive/BTP Project files/forex1.csv', encoding=encoding)
for_test = pd.read_csv('/content/drive/MyDrive/NLP BTP/pegasus.csv')
for_test.head()

,Sentiment,Paraphrased_Text
0,Positive,The euro will benefit from the ECB's determina...
1,Positive,The trend higher may continue.
2,Neutral,Does a jump in EURCHF point to a break in the ...
3,Positive,It is possible that the advance could be exten...
4,Positive,Projections of 10130 for the EURCHF.


In [14]:
import pandas as pd

train_df = for_test[['Sentiment', 'Paraphrased_Text']].rename(columns={'Sentiment': 'Sentiment', 'Paraphrased_Text': 'Text'})


train_df.head()

,Sentiment,Text
0,Positive,The euro will benefit from the ECB's determina...
1,Positive,The trend higher may continue.
2,Neutral,Does a jump in EURCHF point to a break in the ...
3,Positive,It is possible that the advance could be exten...
4,Positive,Projections of 10130 for the EURCHF.


In [15]:
print(train_df)

     Sentiment                                               Text
0     Positive  The euro will benefit from the ECB's determina...
1     Positive                     The trend higher may continue.
2      Neutral  Does a jump in EURCHF point to a break in the ...
3     Positive  It is possible that the advance could be exten...
4     Positive               Projections of 10130 for the EURCHF.
...        ...                                                ...
2286  Positive  Near its highest level since June, the British...
2287  Negative  Credit Suisse predicts that the euro will brea...
2288   Neutral  Near 10950, the shortterm dips are likely to b...
2289  Positive  Looking for an eventual final leg higher to te...
2290  Negative  Wall Street extends losses as theUSDJPY breaks...

[2291 rows x 2 columns]


In [16]:
import pandas as pd

forex_data = pd.read_csv('/content/drive/MyDrive/NLP BTP/sentiment_annotated_with_texts.csv')
forex_data.head()

,published_at,ticker,true_sentiment,title,author,url,source,text,finbert_sentiment,finbert_sent_score
0,2023-01-12 07:47:00,EURCHF,Positive,Euro to benefit from the ECBs pronounced hawki...,FXStreet Insights Team,https://www.fxstreet.com/news/euro-to-benefit-...,FX Street,The Euro was able to appreciate particularly s...,Positive,0.85
1,2023-01-12 10:34:00,EURCHF,Positive,EURCHF Trend higher may remain in place – ING,FXStreet Insights Team,https://www.fxstreet.com/news/eur-chf-trend-hi...,FX Street,EUR/CHF yesterday broke above 1.00. Economists...,Positive,0.51
2,2023-01-12 11:40:00,EURCHF,Neutral,Does a jump in EURCHF point to a break above 1...,FXStreet Insights Team,https://www.fxstreet.com/news/does-a-jump-in-e...,FX Street,EUR/CHF vaults parity for the first time since...,Neutral,0.37
3,2023-01-12 15:32:00,EURCHF,Positive,EURCHF could extend its advance back to levels...,FXStreet Insights Team,https://www.fxstreet.com/news/eur-chf-could-ex...,FX Street,EUR/CHF climbs back above parity. Economists a...,Positive,0.64
4,2023-01-13 11:37:00,EURCHF,Positive,EURCHF to head higher towards 10130 and projec...,FXStreet Insights Team,https://www.fxstreet.com/news/eur-chf-to-head-...,FX Street,EUR/CHF has broken out above the sideways rang...,Positive,0.83


In [17]:
import pandas as pd

test_df = forex_data[['true_sentiment', 'title']].rename(columns={'true_sentiment': 'Sentiment', 'title': 'Text'})


print(test_df)

     Sentiment                                               Text
0     Positive  Euro to benefit from the ECBs pronounced hawki...
1     Positive      EURCHF Trend higher may remain in place – ING
2      Neutral  Does a jump in EURCHF point to a break above 1...
3     Positive  EURCHF could extend its advance back to levels...
4     Positive  EURCHF to head higher towards 10130 and projec...
...        ...                                                ...
2286  Positive  GBPUSD holds steady near its highest level sin...
2287  Negative  EURCHF to break below the 097 level – Credit S...
2288   Neutral  EURUSD Shortterm dips likely supported near 10...
2289  Positive  GBPUSD Looking for an eventual final leg highe...
2290  Negative  USDJPY breaks below 13400 as Wall Street exten...

[2291 rows x 2 columns]


In [18]:
y = test_df['Sentiment'].to_list()

In [10]:
!pip install transformers==4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 24.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1


In [19]:
from transformers import TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import torch

In [21]:
model_name = 'ProsusAI/finbert'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

In [22]:
train_encodings = tokenizer(train_df['Text'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_df['Text'].tolist(), truncation=True, padding=True)

In [ ]:
!pip install accelerate>=0.21.0


In [ ]:
!pip install transformers torch accelerate>=0.21.0


In [ ]:
pip install accelerate -U

In [23]:
from transformers import Trainer, TrainingArguments
import torch
import numpy as np

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, label_mapping):
        self.encodings = encodings
        self.labels = labels
        self.label_mapping = label_mapping

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.label_mapping[self.labels[idx]])  # Map labels to numerical values
        return item

    def __len__(self):
        return len(self.labels)

# Define label mapping
label_mapping = {'Positive': 0, 'Negative': 1, 'Neutral': 2}


train_dataset = CustomDataset(train_encodings, train_df['Sentiment'].tolist(), label_mapping)
test_dataset = CustomDataset(test_encodings, test_df['Sentiment'].tolist(), label_mapping)


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)


predicted_array = []


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


try:

    trainer.train()


    predictions = trainer.predict(test_dataset)


    predicted_labels = np.argmax(predictions.predictions, axis=1)


    reverse_label_mapping = {v: k for k, v in label_mapping.items()}
    predicted_sentiments = [reverse_label_mapping[label] for label in predicted_labels]


    predicted_array.extend(predicted_sentiments)

except Exception as e:
    print("Error during training or evaluation:", e)


print(predicted_array)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2291
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 432


Step,Training Loss
10,1.112900
20,1.176300
30,1.215200
40,1.038800
50,1.007300
60,1.081400
70,1.034900
80,0.984500
90,0.933900
100,0.843100




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 2291
  Batch size = 64


['Negative', 'Neutral', 'Neutral', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Neutral', 'Negative', 'Positive', 'Neutral', 'Positive', 'Negative', 'Positive', 'Positive', 'Negative', 'Positive', 'Negative', 'Neutral', 'Negative', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Neutral', 'Positive', 'Neutral', 'Positive', 'Positive', 'Neutral', 'Positive', 'Negative', 'Negative', 'Negative', 'Negative', 'Neutral', 'Positive', 'Negative', 'Positive', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Positive', 'Negative', 'Positive', 'Negative', 'Negative', 'Neutral', 'Neutral', 'Negative', 'Negative', 'Positive', 'Neutral', 'Neutral', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Positive', 'Positive', 'Neutral', 'Positive', 'Negative', 'Positive', 'Positive', 'Neutral', 'Positive', 'Positive', 'Negative', 'Negative', 'Neutral', 'Negative', 'Negative', 'Positive', 'Negative', 'Neutral', 'Negative', 'Neutral', 'Neutral', 'Negative'

In [25]:
from sklearn.metrics import accuracy_score , classification_report

print(f'Accuracy-Score: {accuracy_score(y, predicted_array)}')

Accuracy-Score: 0.7446529899607158


In [26]:
y_true = test_df['Sentiment']
accuracy = accuracy_score(y_true, predicted_array)
report = classification_report(y_true, predicted_array)
print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')

Accuracy: 0.7446529899607158
Classification Report:
              precision    recall  f1-score   support

    Negative       0.67      0.87      0.76       709
     Neutral       0.77      0.68      0.72       815
    Positive       0.82      0.69      0.75       767

    accuracy                           0.74      2291
   macro avg       0.75      0.75      0.74      2291
weighted avg       0.76      0.74      0.74      2291

